In [85]:
from pathlib import Path
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [87]:
df_train = pd.read_parquet(Path("data") / "train.parquet")

In [89]:
import holidays


def is_fr_bank_holiday(date):
    if date in holidays.France():
        return 1
    else:
        return 0

In [90]:
def is_weekend(day):
    if day == 6 or day == 7:
        return 1
    else:
        return 0

In [92]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour
    X['Weekend'] = X['weekday'].apply(is_weekend)
    X['holiday'] = X['date'].apply(is_fr_bank_holiday)

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [94]:
def _merge_external_data(X):
    file_path = Path('data') / "external_data.csv"
    df_ext = pd.read_csv(file_path, parse_dates=["date"])

    X['date'] = pd.to_datetime(X['date']).astype('datetime64[ns]')
    df_ext['date'] = pd.to_datetime(df_ext['date']).astype('datetime64[ns]')

    X = X.copy()
    # When using merge_asof left frame need to be sorted
    X["orig_index"] = np.arange(X.shape[0])
    X = pd.merge_asof(
        X.sort_values("date"), df_ext[[
            "date", "t", 'pres', 'raf10', 'u', 'ff',
            'vv', 'tend24', 'cod_tend', 'rr3', 'rr12'
        ]].sort_values("date"), on="date"
    )
    # Sort back to the original order
    X = X.sort_values("orig_index")
    del X["orig_index"]
    return X

In [95]:
df_train = _merge_external_data(df_train)

In [97]:
X_dates_encoding = _encode_dates(df_train[["date"]])
df_train = pd.concat([df_train, X_dates_encoding], axis=1)

In [99]:
y = df_train['log_bike_count']
X = df_train.drop('log_bike_count', axis=1)

In [100]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, random_state=2408)

In [103]:
selected_features = [
    'latitude', 't', 'u', 'month', 'weekday', 'hour', 'day', 'pres',
    'raf10', 'ff', 'vv', 'tend24', 'cod_tend', 'rr3', 'rr12'
]
X_train = X_train[selected_features]
X_valid = X_valid[selected_features]

In [104]:
# Initialize and train a XGBRegressor
pipe = XGBRegressor(
    learning_rate=0.22, max_depth=6, n_estimators=600,
    gamma=0.01, min_child_weight=7, reg_alpha=0.15
)
pipe.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.01, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.22, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=7, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=600, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

2:80: E501 line too long (118 > 79 characters)


In [105]:
print(
    f"Train set, RMSE={mean_squared_error(y_train, pipe.predict(X_train), squared=False):.2f}"
)
print(
    f"Test set, RMSE={mean_squared_error(y_valid, pipe.predict(X_valid), squared=False):.2f}"
)

2:80: E501 line too long (94 > 79 characters)
5:80: E501 line too long (93 > 79 characters)


Train set, RMSE=0.49
Test set, RMSE=0.52


In [106]:
df_test = pd.read_parquet(Path("data") / "final_test.parquet")

In [115]:
df_test = _merge_external_data(df_test)

In [116]:
test_dates_encoding = _encode_dates(df_test[["date"]])
df_test = pd.concat([df_test, test_dates_encoding], axis=1)

In [111]:
y_pred = pipe.predict(df_test[selected_features])
results = pd.DataFrame(
    dict(
        Id=np.arange(y_pred.shape[0]),
        log_bike_count=y_pred,
    )
)
results.to_csv("submission.csv", index=False)